In [1]:
import pandas as pd

DAY = 24*60*60

In [2]:
df = pd.read_csv('./connection_graph.csv', index_col=0, low_memory=False)
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383
...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523


In [3]:
# rm the first column and duplicate rows
# df = df.drop(df.columns[0], axis=1)
# df.drop_duplicates(inplace=True)
df
# save to file

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383
...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523


In [4]:
# df.to_csv('./connection_graph.csv')

In [5]:
def get_sec_from_str(time_str):
    time_str = time_str.split(':')
    return (int(time_str[0]) * 3600 + int(time_str[1]) * 60 + int(time_str[2])) % DAY

df['departure_time_sec'] = df['departure_time'].apply(get_sec_from_str)
df['arrival_time_sec'] = df['arrival_time'].apply(get_sec_from_str)
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_sec,arrival_time_sec
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540
...,...,...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543,67080,67140
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808,67140,67260
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584,67260,67320
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523,67320,67380


In [6]:
stops = df['start_stop'].unique()
stops, stops.shape

(array(['Zajezdnia Obornicka', 'Paprotna', 'Obornicka (Wołowska)',
        'Bezpieczna', 'Bałtycka', 'Broniewskiego', 'Pola', 'Syrokomli',
        'Kasprowicza', 'pl. Daniłowskiego', 'Przybyszewskiego',
        'Czajkowskiego', 'KOSZAROWA (Uniwersytet)', 'KOSZAROWA (Szpital)',
        'Berenta', 'KROMERA', 'Mosty Warszawskie', 'Wyszyńskiego',
        'Ogród Botaniczny', 'Katedra',
        'Urząd Wojewódzki (Muzeum Narodowe)', 'Poczta Główna',
        'skwer Krasińskiego', 'Wzgórze Partyzantów', 'Renoma',
        'Arkady (Capitol)', 'Pl. Hirszfelda', 'Krucza',
        'Krucza (Mielecka)', 'Inżynierska', 'Aleja Pracy', 'FAT',
        'GRABISZYŃSKA (Cmentarz)', 'Solskiego', 'Wiejska', 'Kadłubka',
        'Stanki', 'Bukowskiego', 'RACŁAWICKA', 'Rymarska', 'Wawrzyniaka',
        'Chłodna', 'Sowia', 'KRZYKI', 'Dworzec Główny (Dworcowa)',
        'Krasińskiego', 'Damrota', 'Koszarowa', 'Sołtysowicka',
        'Poprzeczna', 'Redycka', 'Bagatela', 'SOŁTYSOWICE', 'POŚWIĘTNE',
        'Wołowska',

In [13]:
import numpy as np

import networkx as nx

graph = nx.DiGraph()
for stop in stops:
    graph.add_node(stop)

for number, edge in df.iterrows():
  if not graph.has_edge(edge['start_stop'], edge['end_stop']):
    graph.add_edge(edge['start_stop'], edge['end_stop'], schedule=[])

  graph[edge['start_stop']] [edge['end_stop']] ['schedule'].append({
    # 'dep': edge['departure_time_sec'],
    'end_stop_lat': edge['end_stop_lat'],
    'end_stop_lon': edge['end_stop_lon'],
    'arrival': edge['arrival_time_sec'],
    'id': number
    })


def convert_to_dataframe(graph):
  for start, stop in graph.edges:
    graph[start][stop]['schedule'] = pd.DataFrame(graph[start][stop]['schedule'])

convert_to_dataframe(graph)

In [8]:
def print_graph(graph):
  for start, stop in graph.edges:
    print(start, '->', stop)
    print(graph[start][stop]['schedule']['id'])


In [9]:
from typing import Dict

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge: float, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time
      return id

  def get_best_time(self, edge, departure):
    schedule = edge['schedule']
    times = schedule[['arrival']]

    best_time = times[times['arrival'] > departure].min().iloc[0]

    try:
      idx = times[times['arrival'] > departure].idxmin()
      row = schedule['id'][idx]
    except:
      row = None

    return best_time, row

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]


np.seterr(divide='ignore', invalid='ignore')

def djikstra(graph: nx.Graph, start: str, destination: str, starting_time: int):
  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    best_neighbour = NodeFeatures(-1, "", np.inf, None)
    for neigbour in neighbours:
      if not neigbour in unvisited:
        continue

      unvisited[neigbour].set_time(start_stop.time, neighbours[neigbour], start_stop)

      if best_neighbour.time > unvisited[neigbour].time:
        best_neighbour = unvisited[neigbour]

    if best_neighbour.time == np.inf:
      start_stop = unvisited.popitem()[1]
      continue

    visited[start_stop.name] = start_stop
    start_stop = unvisited.pop(best_neighbour.name)

  return get_best_path(visited, destination)


visited = djikstra(graph, 'DWORZEC AUTOBUSOWY', 'FAT', 34900)
visited

In [10]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

node = None
for idx, node in enumerate(visited):
  # print(node.name, node.time)
  try:
    row = df.loc[node.id]
    # display only the most important columns: line, start_stop, end_stop, departure_time, arrival_time
    print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].values)

  except:...
    # print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].columns.values)
  print()


[['125' 'DWORZEC AUTOBUSOWY' 'EPI' '09:41:00' '09:44:00']]

[['125' 'EPI' 'Arena' '09:44:00' '09:45:00']]

[['125' 'Arena' 'Komandorska' '09:45:00' '09:46:00']]

[['125' 'Komandorska' 'Pl. Hirszfelda' '09:46:00' '09:49:00']]

[['125' 'Pl. Hirszfelda' 'Krucza' '09:49:00' '09:51:00']]

[['144' 'Krucza' 'Gajowicka' '09:50:00' '09:52:00']]

[['20' 'Gajowicka' 'Mielecka' '09:54:00' '09:55:00']]

[['20' 'Mielecka' 'Ojca Beyzyma' '09:55:00' '09:56:00']]

[['136' 'Ojca Beyzyma' 'Aleja Pracy' '09:56:00' '09:57:00']]

[['136' 'Aleja Pracy' 'FAT' '09:57:00' '09:59:00']]



In [11]:
departure = 34900

df2 = pd.DataFrame(0, index=np.arange(df.shape[0]), columns=['diff'])
df2['diff'] = (df['departure_time_sec'] + DAY - departure) % DAY

# df2['diff'].min() find min and index of min
df2[df2['diff'] > 0].idxmin().iloc[0]

324

In [21]:
from typing import Dict

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge: float, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time
      return id

  def get_best_time(self, edge, departure):
    schedule = edge['schedule']
    times = schedule[['arrival']]

    distance_cost = calculate_manhatan(schedule['end_stop_lat'], schedule['end_stop_lon'])
    best_time = times[times['arrival'] - distance_cost > departure].min().iloc[0]

    try:
      idx = times[times['arrival'] - distance_cost > departure].idxmin()
      row = schedule['id'][idx]
    except:
      row = None

    return best_time, row

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED
  return (abs(start_lat - target_lat) + abs(start_long - target_long)) / TRAM_SPEED

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16 * 1000 / 3600
target_lat = None
target_lat = None

def a_star(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    best_neighbour = NodeFeatures(-1, "", np.inf, None)
    for neigbour in neighbours:
      if not neigbour in unvisited:
        continue

      unvisited[neigbour].set_time(start_stop.time, neighbours[neigbour], start_stop)

      if best_neighbour.time > unvisited[neigbour].time:
        best_neighbour = unvisited[neigbour]

    if best_neighbour.time == np.inf:
      start_stop = unvisited.popitem()[1]
      continue

    visited[start_stop.name] = start_stop
    start_stop = unvisited.pop(best_neighbour.name)

  return get_best_path(visited, destination)


visited = a_star(graph, 'DWORZEC AUTOBUSOWY', 'FAT', 34900)
visited

In [22]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

node = None
for idx, node in enumerate(visited):
  # print(node.name, node.time)
  try:
    row = df.loc[node.id]
    # display only the most important columns: line, start_stop, end_stop, departure_time, arrival_time
    print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].values)

  except:...
    # print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].columns.values)
  print()


[['125' 'DWORZEC AUTOBUSOWY' 'EPI' '09:41:00' '09:44:00']]

[['125' 'EPI' 'Arena' '09:44:00' '09:45:00']]

[['125' 'Arena' 'Komandorska' '09:45:00' '09:46:00']]

[['125' 'Komandorska' 'Pl. Hirszfelda' '09:46:00' '09:49:00']]

[['125' 'Pl. Hirszfelda' 'Krucza' '09:49:00' '09:51:00']]

[['144' 'Krucza' 'Gajowicka' '09:50:00' '09:52:00']]

[['20' 'Gajowicka' 'Mielecka' '09:54:00' '09:55:00']]

[['20' 'Mielecka' 'Ojca Beyzyma' '09:55:00' '09:56:00']]

[['136' 'Ojca Beyzyma' 'Aleja Pracy' '09:56:00' '09:57:00']]

[['136' 'Aleja Pracy' 'FAT' '09:57:00' '09:59:00']]

